In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import filecmp
import json
import nilearn
import nilearn.image
import fsl
import fsl.wrappers
import fsl.utils.image.resample
from fsl.data.image import Image
df = pd.read_pickle('df.pkl')

In [ ]:
# CHANGE THIS VARIABLE TO REFLECT WHERE YOUR HBN BIDS DATA LIVE ()
bids_dir = '/om4/group/gablab/data/HBN/' # Path should end with a '/'
tractseg_dir = derivatives_dir+'TractSeg_NEW/'

# TRACTS TO ANALYZE
tracts = ['AF_left','AF_right','SLF_I_left','SLF_I_right','SLF_II_left','SLF_II_right',
         'SLF_III_left','SLF_III_right','ILF_left','ILF_right','IFO_left','IFO_right',
         'UF_left','UF_right','SCP_left','SCP_right','ICP_left','ICP_right','MCP','CC_7']


In [ ]:
subs = df['subjects']
tract_vol_proportions = {}
tracts = ['AF_left','AF_right','SLF_I_left','SLF_I_right','SLF_II_left','SLF_II_right',
         'SLF_III_left','SLF_III_right','ILF_left','ILF_right','IFO_left','IFO_right',
         'UF_left','UF_right','SCP_left','SCP_right','ICP_left','ICP_right','MCP','CC_7']

for tract in tracts:
    tract_vol_proportions[tract] = []
    
for sub in subs:
    # Load the white mattter and get number of white matter voxels
    wm_mask_path = tractseg_dir+sub+'/dwi/'+sub+'_space-reorient_label-WM_mask.nii.gz'
    wm_mask_img = Image(wm_mask_path)
    wm_vol = sum(sum(sum(wm_mask_img.data)))
    
    # Resample tract mask to anatomical space and get proportion of tract voxels to white matter voxels
    for tract in tracts:
        tract_mask_path = tractseg_dir+sub+'/bundle_segmentations/'+tract+'.nii.gz'
        tract_mask_img = Image(tract_mask_path)
        tract_mask_resampled = fsl.utils.image.resample.resampleToReference(tract_mask_img, wm_mask_img)[0]
        tract_vol = sum(sum(sum(tract_mask_resampled)))
        tract_vol_proportions[tract].append(tract_vol/wm_vol)

np.save('tract_vols.npy',tract_vols_proportions)